In [1]:
#!pip install tika
#!pip install pypdftk
from tika import parser 
import uuid
from os import listdir
from os.path import isfile, join
import pandas as pd

In [2]:
# set basics

Title = 'blank'
Text = "blank"
Journal_information = 'blank'
Published = 'unknown'
PMID = 'unknown'
DOI = uuid.uuid4()
Institution = 'unknown'

In [3]:
website_organisations = ['mckinsey', 'hse', 'pwc', 'kpmg', 
                        'ey', 'oecd','deloitte',
                        'bls.gov','europarl','unido','who']

In [4]:
def check_http(text):
    if 'http' in text:
        return True
    else:
        return False


In [5]:
def remove_https(text):     
    
    if 'http' in text:
        #sprint('initial loop')

        # find start of https
        html_start = text.find('http')
        temp = text[html_start:]
        space_after_html = temp.find(" ")

        if space_after_html == -1:
            space_after_html = len(text)

        # start of text to html    #loction of first " after html to end"
        result = text[:html_start]+text[html_start+space_after_html:]

        # recursive

        if 'http' in result:
            result = remove_https(result)                       
            return result
        else:
            return result

    else:
        return text

            

In [6]:
def remove_www(text):     
    
    if 'www' in text:
        #sprint('initial loop')

        # find start of https
        html_start = text.find('www')
        temp = text[html_start:]
        space_after_html = temp.find(" ")

        if space_after_html == -1:
            space_after_html = len(text)

        # start of text to html    #loction of first " after html to end"
        result = text[:html_start]+text[html_start+space_after_html:]

        # recursive

        if 'www' in result:
            result = remove_https(result)                       
            return result
        else:
            return result

    else:
        return text

            

In [7]:
def remove_mailto(text):     
    
    if 'mailto' in text:
        #sprint('initial loop')

        # find start of https
        html_start = text.find('mailto')
        temp = text[html_start:]
        space_after_html = temp.find(" ")


        if space_after_html == -1:
            space_after_html = len(text)

        # start of text to html    #loction of first " after html to end"
        result = text[:html_start]+text[html_start+space_after_html:]

        # recursive

        if 'mailto' in result:

            result = remove_https(result)                       
            return result
        else:
            return result

    else:
        return text

            

In [8]:
folder_path = "C:/Users\Declan/Desktop/downloaded_pdfs/"


pdf_files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

output_list = []

for pdf_file in pdf_files:
    pdf_file_path = folder_path+pdf_file
    
    
    Title = 'blank'
    Text = "blank"
    Journal_information = 'blank'
    Published = 'unknown'
    PMID = 'unknown'
    DOI = uuid.uuid4()
    Institution = 'unknown'
    
    # do parsing here
    
    
    Title = str(pdf_file)
    Title = Title[Title.find("-")+1:]
    
    metadata = parser.from_file(pdf_file_path)
    
    Text = metadata['content'].replace("\n","")
    Text = Text.strip().replace('..',"").replace('\xa0'," ").lower()
    Text = remove_https(Text)
    Text = remove_www(Text)
    Text = remove_mailto(Text)

    Text = Text.replace('deloitte touche tohmatsu'," ")
    Text = Text.replace('mckinsey & company', " ")
    Text = Text.replace('mckinsey and company', " ")
    Text = Text.replace("all rights reserved", " ")
    Text = " ".join([word for word in Text.split() if '.com' not in word])
    Text = " ".join([word for word in Text.split() if '.www' not in word])
    Text = " ".join([word for word in Text.split() if 'copyright' not in word])
    Text = " ".join([word for word in Text.split() if '©' not in word])
    
    Text = Text.split()[50:len(Text.split())-300] 
    Text = " ".join([word for word in Text])

    Journal_information = str(pdf_file)
    Journal_information = Journal_information[:Journal_information.find('-')]

    try:
        Published = metadata['metadata']['Last-Modified'][:4] # can get date as well
    except:
        Published = 'unknown'
    
    Institution = Journal_information
    
    
    pdf_info = {
            'Title': Title,
            'Text': str(Text),
            'Metadata':str(metadata),
            "Journal Information": Journal_information,
            'Published':Published,
            'Year' : Published,
            'PMID':PMID, 
            'DOI':str(DOI),
            'Institutions':Journal_information,
            'Source Type': 'grey'
            }
    

    
    # append
    output_list.append(pdf_info)

In [9]:
df_final = pd.DataFrame.from_dict(output_list)

In [10]:
df_final

,Title,Text,Metadata,Journal Information,Published,Year,PMID,DOI,Institutions
0,au-audit-clarity-disclosure-climate-related-ri...,"regulators, and continued social pressure in a...","{'metadata': {'Author': 'Prajita Balavinodan',...",deloitte,2020,2020,unknown,c674789c-c04d-4489-bef2-61717c6bee56,deloitte
1,au-hc-re-architecting-work-models-111120.pdf,philosophies 12 five watchpoints to consider w...,"{'metadata': {'Author': 'Gopal, Arun (US - Hyd...",deloitte,2020,2020,unknown,169d1385-a430-4f07-8a91-0bbab199d61e,deloitte
2,DI_2021-Global-Marketing-Trends_US.pdf,"across the globe, deloitte digital helps clien...",{'metadata': {'Content-Type': 'application/pdf...,deloitte,2020,2020,unknown,ced797ab-c6c4-4c62-bce4-0e0bd7b0167c,deloitte
3,DI_2021-Tech-Trends.pdf,technologyconnectivityof tomorrowarchitecture ...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2021,2021,unknown,33e10b0f-7acf-438d-8988-a02a0bb8a03c,deloitte
4,di_human-capital-trends.pdf,capital area of to learn more. the social ente...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2020,2020,unknown,d8f30b67-af59-4b83-a533-3667c2dc81ce,deloitte
5,gx-eri-future-of-energy.pdf,clear is that we’re well on our way to a new e...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2020,2020,unknown,222679b2-5b61-48d5-9b2f-09458cb64c36,deloitte
6,gx-respond-tal-strat-review-employment-contrac...,enhance employee experience by providing clari...,"{'metadata': {'Author': 'Urbanowicz, Janek', '...",deloitte,2020,2020,unknown,9002603e-8716-4269-bc3c-37e3c7e681a1,deloitte
7,recognising-emerging-risks-and-opportunities.pdf,"techniques to systematically identify, priorit...","{'metadata': {'Author': 'Akshay Manjrekar', 'C...",deloitte,2021,2021,unknown,920416c7-ff6d-4a84-92e8-984d588fc89c,deloitte
8,uk-future-creative-economy-report-final.pdf,of the scope and limitations set out below.the...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2021,2021,unknown,357b8451-3078-4045-ac24-4a623ccbb6ef,deloitte
9,Trends-Business-models-2030.pdf,fdi foreign direct investmentghc green house g...,{'metadata': {'Content-Type': 'application/pdf...,emetris,2016,2016,unknown,347728df-5f18-42e8-98eb-01763d1db29d,emetris


In [11]:
file = df_final.to_json("C:/Users/Declan/Desktop/output.json")

In [12]:
df_final

,Title,Text,Metadata,Journal Information,Published,Year,PMID,DOI,Institutions
0,au-audit-clarity-disclosure-climate-related-ri...,"regulators, and continued social pressure in a...","{'metadata': {'Author': 'Prajita Balavinodan',...",deloitte,2020,2020,unknown,c674789c-c04d-4489-bef2-61717c6bee56,deloitte
1,au-hc-re-architecting-work-models-111120.pdf,philosophies 12 five watchpoints to consider w...,"{'metadata': {'Author': 'Gopal, Arun (US - Hyd...",deloitte,2020,2020,unknown,169d1385-a430-4f07-8a91-0bbab199d61e,deloitte
2,DI_2021-Global-Marketing-Trends_US.pdf,"across the globe, deloitte digital helps clien...",{'metadata': {'Content-Type': 'application/pdf...,deloitte,2020,2020,unknown,ced797ab-c6c4-4c62-bce4-0e0bd7b0167c,deloitte
3,DI_2021-Tech-Trends.pdf,technologyconnectivityof tomorrowarchitecture ...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2021,2021,unknown,33e10b0f-7acf-438d-8988-a02a0bb8a03c,deloitte
4,di_human-capital-trends.pdf,capital area of to learn more. the social ente...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2020,2020,unknown,d8f30b67-af59-4b83-a533-3667c2dc81ce,deloitte
5,gx-eri-future-of-energy.pdf,clear is that we’re well on our way to a new e...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2020,2020,unknown,222679b2-5b61-48d5-9b2f-09458cb64c36,deloitte
6,gx-respond-tal-strat-review-employment-contrac...,enhance employee experience by providing clari...,"{'metadata': {'Author': 'Urbanowicz, Janek', '...",deloitte,2020,2020,unknown,9002603e-8716-4269-bc3c-37e3c7e681a1,deloitte
7,recognising-emerging-risks-and-opportunities.pdf,"techniques to systematically identify, priorit...","{'metadata': {'Author': 'Akshay Manjrekar', 'C...",deloitte,2021,2021,unknown,920416c7-ff6d-4a84-92e8-984d588fc89c,deloitte
8,uk-future-creative-economy-report-final.pdf,of the scope and limitations set out below.the...,{'metadata': {'Content-Type': 'application/pdf...,deloitte,2021,2021,unknown,357b8451-3078-4045-ac24-4a623ccbb6ef,deloitte
9,Trends-Business-models-2030.pdf,fdi foreign direct investmentghc green house g...,{'metadata': {'Content-Type': 'application/pdf...,emetris,2016,2016,unknown,347728df-5f18-42e8-98eb-01763d1db29d,emetris


In [13]:
#str(df_final['Institutions'][51])+"-"+str(df_final['Title'][51])

In [14]:
len(df_final['Text'][0])

22724

In [ ]:
list1 = Text_.split()[50:450]#len(Text_.split())-300] 
list1 = " ".join([word for word in list1])
list1

In [ ]:
len(list1.split())